In [0]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.7 gigabytes of available RAM

To enable a high-RAM runtime, select the Runtime > "Change runtime type"
menu, and then select High-RAM in the Runtime shape dropdown. Then, 
re-execute this cell.


In [0]:
import csv
import numpy as np
from keras.models import Model
from keras.layers import Input, LSTM, Dense
from keras.models import load_model

In [0]:
START = "^"
END = "$"
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
num_samples = 10000  # Number of samples to train on.
latent_dim = 256
DATA_PATH = './drive/My Drive/Colab Notebooks/12-data'

def get_file_data(path):
    input_texts, target_texts = [], []
    with open(path) as f:
        reader = csv.reader(f, dialect='excel-tab')
        for i, row in enumerate(reader):
            src = START + row[1] + END
            tgt = START + row[2] + END
            if i < num_samples:
                input_texts.append(src)
                # input_texts.append(tgt)
                target_texts.append(tgt)
                # target_texts.append(src)
    return input_texts, target_texts

In [0]:
input_texts, target_texts = get_file_data(f'{DATA_PATH}/en-train-100K.txt')
# dev_src, dev_target = get_file_data(f'{DATA_PATH}/en-dev.txt')
# test_src, test_target = get_file_data(f'{DATA_PATH}/en-test.txt')

In [78]:
print(len(input_texts))
print(input_texts[:2])
print(target_texts[:2])

10000
['^Jumby now wants to be born .$', '^It was a difficult and long delivery .$']
['^Jumby want birth .$', '^The delivery was difficult and long .$']


In [0]:
def get_chars(input_texts, target_texts):
    input_characters = set()
    target_characters = set()

    for input_text in input_texts:
        for char in input_text:
          if char not in input_characters:
              input_characters.add(char)

    for target_text in target_texts:
      for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

    input_characters = sorted(list(input_characters))
    target_characters = sorted(list(target_characters))

    return input_characters, target_characters

In [80]:
input_characters, target_characters = get_chars(input_texts, target_texts)
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 82
Number of unique output tokens: 101
Max sequence length for inputs: 123
Max sequence length for outputs: 126


In [0]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

In [0]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [83]:
print('Encoder input shape:', encoder_input_data.shape)
print('Decoder input shape:', decoder_input_data.shape)
print('Decoder taget shape:', decoder_target_data.shape)

Encoder input shape: (10000, 123, 82)
Decoder input shape: (10000, 126, 101)
Decoder taget shape: (10000, 126, 101)


In [0]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.

In [0]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [0]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 113s 14ms/step - loss: 0.6754 - accuracy: 0.8485 - val_loss: 0.5657 - val_accuracy: 0.8589
Epoch 2/100
8000/8000 [==============================] - 114s 14ms/step - loss: 0.4830 - accuracy: 0.8715 - val_loss: 0.4176 - val_accuracy: 0.8796
Epoch 3/100
8000/8000 [==============================] - 112s 14ms/step - loss: 0.4002 - accuracy: 0.8920 - val_loss: 0.3604 - val_accuracy: 0.8980
Epoch 4/100
8000/8000 [==============================] - 127s 16ms/step - loss: 0.3600 - accuracy: 0.8983 - val_loss: 0.3390 - val_accuracy: 0.9012
Epoch 5/100
8000/8000 [==============================] - 111s 14ms/step - loss: 0.3397 - accuracy: 0.9029 - val_loss: 0.3234 - val_accuracy: 0.9054
Epoch 6/100
8000/8000 [==============================] - 110s 14ms/step - loss: 0.3231 - accuracy: 0.9070 - val_loss: 0.3081 - val_accuracy: 0.9111
Epoch 7/100
8000/8000 [==============================] - 111s 14

In [0]:
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [0]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index[START]] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == END or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [102]:
for seq_index in range(10):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: ^Jumby now wants to be born .$
Decoded sentence: .ö.Ìö::::Ó:Óiiii¹¹~´BMMMccDDQcÚDD??????????????????????????????????????????????????????????????????????????????????????????????
-
Input sentence: ^It was a difficult and long delivery .$
Decoded sentence: .ö.Ìö::::Ó:Óiiii¹¹~´BMMMccDDQcÚDD??????????????????????????????????????????????????????????????????????????????????????????????
-
Input sentence: ^I like to be beautiful everyday .$
Decoded sentence: .ö.Ìö::::Ó:Óiiii¹¹~´BMMMccDDQcÚDD??????????????????????????????????????????????????????????????????????????????????????????????
-
Input sentence: ^Bernadette wants a prenup .$
Decoded sentence: .ö.Ìö::::Ó:Óiiii¹¹~´BMMMccDDQcÚDD??????????????????????????????????????????????????????????????????????????????????????????????
-
Input sentence: ^Don 't say you don 't remember me .$
Decoded sentence: .ö.Ìö::::Ó:Óiiii¹¹~´BMMMccDDQcÚDD??????????????????????????????????????????????????????????????????????????????????????????????
-
In

In [0]:
### eval

In [0]:
model = load_model(f'{DATA_PATH}/s2s.h5')

In [8]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 82)     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 101)    0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 256), (None, 347136      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 256),  366592      input_2[0][0]                    
                                                                 lstm_1[0][1]               

In [10]:
!pip install py-rouge
import rouge

     |████████████████████████████████| 61kB 2.1MB/s 


In [13]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
def prepare_results(p, r, f):
    return '\t{}:\t{}: {:5.2f}\t{}: {:5.2f}\t{}: {:5.2f}'.format(metric, 'P', 100.0 * p, 'R', 100.0 * r, 'F1', 100.0 * f)


def evaluate(hypothesis, references):
  for aggregator in ['Avg', 'Best', 'Individual']:
      print('Evaluation with {}'.format(aggregator))
      apply_avg = aggregator == 'Avg'
      apply_best = aggregator == 'Best'

      evaluator = rouge.Rouge(metrics=['rouge-n', 'rouge-l', 'rouge-w'],
                            max_n=4,
                            limit_length=True,
                            length_limit=100,
                            length_limit_type='words',
                            apply_avg=apply_avg,
                            apply_best=apply_best,
                            alpha=0.5, # Default F1_score
                            weight_factor=1.2,
                            stemming=True)

      scores = evaluator.get_scores(hypothesis, references)

      for metric, results in sorted(scores.items(), key=lambda x: x[0]):
          if not apply_avg and not apply_best: # value is a type of list as we evaluate each summary vs each reference
              for hypothesis_id, results_per_ref in enumerate(results):
                  nb_references = len(results_per_ref['p'])
                  for reference_id in range(nb_references):
                      print('\tHypothesis #{} & Reference #{}: '.format(hypothesis_id, reference_id))
                      print('\t' + prepare_results(results_per_ref['p'][reference_id], results_per_ref['r'][reference_id], results_per_ref['f'][reference_id]))
              print()
          else:
              print(prepare_results(results['p'], results['r'], results['f']))
      print()

In [0]:
def get_evaluation_data():
  hypothesis, references = [], []
  for seq_index in range(100):
      # Take one sequence (part of the training set)
      # for trying out decoding.
      input_seq = encoder_input_data[seq_index: seq_index + 1]
      decoded_sentence = decode_sequence(input_seq)
      hypothesis.append(train_src[seq_index])
      references.append(decoded_sentence)

  return hypothesis, references

In [0]:
hypothesis, references = get_evaluation_data()

In [32]:
evaluate(hypothesis, references)

Evaluation with Avg
	rouge-w:	P:  0.00	R:  0.00	F1:  0.00
	rouge-w:	P:  0.00	R:  0.00	F1:  0.00
	rouge-w:	P:  0.00	R:  0.00	F1:  0.00
	rouge-w:	P:  0.00	R:  0.00	F1:  0.00
	rouge-w:	P:  0.00	R:  0.00	F1:  0.00
	rouge-w:	P:  0.00	R:  0.00	F1:  0.00

Evaluation with Best
	rouge-w:	P:  0.00	R:  0.00	F1:  0.00
	rouge-w:	P:  0.00	R:  0.00	F1:  0.00
	rouge-w:	P:  0.00	R:  0.00	F1:  0.00
	rouge-w:	P:  0.00	R:  0.00	F1:  0.00
	rouge-w:	P:  0.00	R:  0.00	F1:  0.00
	rouge-w:	P:  0.00	R:  0.00	F1:  0.00

Evaluation with Individual
	Hypothesis #0 & Reference #0: 
		rouge-w:	P:  0.00	R:  0.00	F1:  0.00
	Hypothesis #1 & Reference #0: 
		rouge-w:	P:  0.00	R:  0.00	F1:  0.00
	Hypothesis #2 & Reference #0: 
		rouge-w:	P:  0.00	R:  0.00	F1:  0.00
	Hypothesis #3 & Reference #0: 
		rouge-w:	P:  0.00	R:  0.00	F1:  0.00
	Hypothesis #4 & Reference #0: 
		rouge-w:	P:  0.00	R:  0.00	F1:  0.00
	Hypothesis #5 & Reference #0: 
		rouge-w:	P:  0.00	R:  0.00	F1:  0.00
	Hypothesis #6 & Reference #0: 
		rouge-w:	P:  0